# imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, ResNet50, MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt

# Compile model

In [ ]:
def create_transfer_learning_model(base_model_name='EfficientNetB0', 
                                 input_shape=(256, 256, 3), 
                                 num_classes=9,
                                 freeze_base=True):
    """
    Create a transfer learning model using ImageNet pre-trained weights
    
    Args:
        base_model_name: Choice of base model ('EfficientNetB0', 'ResNet50', 'MobileNetV2')
        input_shape: Input image shape
        num_classes: Number of classes to classify
        freeze_base: Whether to freeze base model weights initially
    
    Returns:
        Compiled Keras model
    """
    
    # Choose base model
    if base_model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(
            weights='imagenet',  # Use ImageNet weights
            include_top=False,   # Don't include final classification layer
            input_shape=input_shape
        )
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    elif base_model_name == 'MobileNetV2':
        base_model = MobileNetV2(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    else:
        raise ValueError("Choose from 'EfficientNetB0', 'ResNet50', or 'MobileNetV2'")
    
    # Freeze base model layers initially
    if freeze_base:
        base_model.trainable = False
        print(f"Base model ({base_model_name}) frozen - {len(base_model.layers)} layers")
    else:
        print(f"Base model ({base_model_name}) unfrozen - {len(base_model.layers)} layers")
    
    # Add custom classification head
    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        
        # First dense layer
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        # Second dense layer
        Dense(256, activation='relu'), 
        BatchNormalization(),
        Dropout(0.3),
        
        # Output layer
        Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model


In [ ]:
def compile_transfer_model(model, learning_rate=0.001):
    """Compile the transfer learning model"""
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_3_accuracy']
    )
    return model


In [ ]:
def create_callbacks(model_name='best_transfer_model.keras'):
    """Create training callbacks"""
    return [
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            model_name,
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]

# Compare models

In [ ]:
# Create and compare different models
def compare_models(train_generator, val_generator, class_weights):
    """Compare different pre-trained models"""
    
    models_to_test = ['EfficientNetB0', 'ResNet50', 'MobileNetV2']
    results = {}
    
    for model_name in models_to_test:
        print(f"\n{'='*50}")
        print(f"Training {model_name}")
        print(f"{'='*50}")
        
        # Create model
        model, base_model = create_transfer_learning_model(
            base_model_name=model_name,
            input_shape=(256, 256, 3),
            num_classes=9,
            freeze_base=True
        )
        
        # Compile model
        model = compile_transfer_model(model, learning_rate=0.001)
        
        # Print model summary
        print(f"\nModel Summary for {model_name}:")
        model.summary()
        
        # Train model
        callbacks = create_callbacks(f'best_{model_name.lower()}_model.keras')
        
        history = model.fit(
            train_generator,
            epochs=30,  # Start with fewer epochs for transfer learning
            validation_data=val_generator,
            class_weight=class_weights,
            callbacks=callbacks,
            verbose=1
        )
        
        # Evaluate
        val_loss, val_accuracy, val_top3 = model.evaluate(val_generator, verbose=0)
        
        results[model_name] = {
            'model': model,
            'base_model': base_model,
            'history': history,
            'val_accuracy': val_accuracy,
            'val_loss': val_loss,
            'val_top3': val_top3
        }
        
        print(f"\n{model_name} Results:")
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Top-3 Accuracy: {val_top3:.4f}")
    
    return results

# Fine tunning chosen model

In [ ]:
# Fine-tuning function (advanced technique)
def fine_tune_model(model, base_model, train_generator, val_generator, 
                   class_weights, fine_tune_at=100):
    """
    Fine-tune the model by unfreezing some layers of the base model
    
    Args:
        model: The trained model
        base_model: The base pre-trained model
        fine_tune_at: Layer index from which to start fine-tuning
    """
    
    print(f"\nFine-tuning from layer {fine_tune_at}...")
    
    # Unfreeze the top layers of the model
    base_model.trainable = True
    
    # Freeze all layers except the top ones
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Recompile with lower learning rate for fine-tuning
    model.compile(
        optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_3_accuracy']
    )
    
    print(f"Trainable layers: {sum([1 for layer in model.layers if layer.trainable])}")
    
    # Continue training
    fine_tune_callbacks = create_callbacks('fine_tuned_model.h5')
    
    fine_tune_history = model.fit(
        train_generator,
        epochs=20,  # Fewer epochs for fine-tuning
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=fine_tune_callbacks,
        verbose=1
    )
    
    return fine_tune_history

# Run chosen model

In [ ]:
# Create the model (EfficientNetB0 is recommended for your use case)
model, base_model = create_transfer_learning_model(
    base_model_name='EfficientNetB0',
    input_shape=(256, 256, 3),
    num_classes=9,
    freeze_base=True
)

# Compile the model
model = compile_transfer_model(model, learning_rate=0.001)

# Print model info
print(f"\nModel Architecture:")
model.summary()

print(f"\nTotal parameters: {model.count_params():,}")
trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {model.count_params() - trainable_params:,}")

# Training setup
callbacks = create_callbacks('best_efficientnet_model.keras')

In [ ]:
# Train transfer learning model
history = model.fit(
    train_generator,
    epochs=50,  # Usually needs fewer epochs than training from scratch
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# After training, optionally fine-tune
# fine_tune_history = fine_tune_model(model, base_model, train_generator, val_generator, class_weights)

# Plot results

In [ ]:
# Plot training history
def plot_transfer_history(history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation') 
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid()
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Training')
    axes[0, 1].plot(history.history['val_loss'], label='Validation')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].legend()
    axes[0, 1].grid()
    
    # Top-3 Accuracy
    axes[1, 0].plot(history.history['top_3_accuracy'], label='Training')
    axes[1, 0].plot(history.history['val_top_3_accuracy'], label='Validation')
    axes[1, 0].set_title('Top-3 Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid()
    
    # Learning Rate
    if 'lr' in history.history:
        axes[1, 1].plot(history.history['lr'])
        axes[1, 1].set_title('Learning Rate')
        axes[1, 1].set_yscale('log')
        axes[1, 1].grid()
    
    plt.tight_layout()
    plt.show()